## Install libraries

In [ ]:
#!pip install pymorphy2
#!python -m nltk.downloader all
#!pip install sentence_transformers
#!pip install sentence_transformers
#!pip install transformers
#!pip install nltk

## Import libraries

In [23]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
import string 
import nltk
import pymorphy2
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk import word_tokenize
import pickle
import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
#from deep_translator import GoogleTranslator
tqdm.pandas()

In [ ]:
import os
import sys 
sys.path.append(os.path.abspath("./milib/"))

from preprocessing import add_punctuation, specific_cleaning, html_cleaning, preprocessing
from models_fit_save_load import tfidf_fit_save, tfidf_load_model, ruBERT_download_save
from models_fit_save_load import ruBERT_load, LaBSE_load, make_embeddings
from recomendations import make_tfidf_recomendation, make_recomendation, make_labse_recomendation

## Download data

In [13]:
#data = pd.read_csv('../data/data_full_translation.csv')
#data_preproc = pd.read_csv('../data/data_full_translation_preprocess.csv')

In [ ]:
data_orig = pd.read_csv('../data/data_orig.csv')
del_index = np.load('../data/del_index')
data_orig = data_orig.drop(index=del_index)
del_ind = data_orig[data_orig['description'].apply(lambda x: len(x)<21 if type(x)==str else True)][data_orig['description'].notna()].index
data_orig.loc[del_ind,'description'] = np.nan


## Preprocessing

Removing misleading rows based on preliminary data analysis.

In [ ]:
#data = specific_cleaning(dataframe)

Delete most frequent html symbols

In [26]:
data['title'] = data['title'].apply(lambda x: html_cleaning(x) if type(x)==str else x)
data['description'] = data['description'].apply(lambda x: html_cleaning(x) if type(x)==str else x)

Add dot at the end of the text if there are no punctuation marks in it.

In [27]:
data['title'] = add_punctuation(data['title'])
data['description'] = add_punctuation(data['description'])

Translate text with english letters.

In [ ]:
def translate(text):
    return GoogleTranslator(source='en', target='ru').translate(GoogleTranslator(source='ru', target='en').translate(seq))

error_list = []
for i in tqdm(np.arange(0,len(data))):
    seq = data.loc[i,'title']
    if type(seq) == str and search_eng(seq):
       try: 
         text = translate(seq)
         data.loc[i,'title'] = text
       except:
         error_list += [i]

In [ ]:
def translate(text):
    return GoogleTranslator(source='en', target='ru').translate(GoogleTranslator(source='ru', target='en').translate(seq))


for i in tqdm(np.arange(0,len(data))):
    seq = data.loc[i,'description']
    if type(seq) == str and search_eng(seq):
       try: 
         text = translate(seq)
         data.loc[i,'description'] = text
       except:
         error_list += [i]

Rows witn index in error_list should be translate. 

In [ ]:
error_list

Text preprocessing: lowing, lemmatization, stemming

In [ ]:
data['title'] = data['title'].progress_apply(lambda x: preprocessing(x) if type(x)==str else x)
data['description'] = data['description'].progress_apply(lambda x: preprocessing(x) if type(x)==str else x)

Combine the title and description together

In [28]:
data['description'] = data['description'].apply(lambda x: x if type(x)==str else '')
data['full_text'] = data['title'] + ' ' + data['description']

## Approaches

### 1. TF-IDF

Creating models and matrices for statistical analysis (tf-idf)

In [ ]:
tfidf_fit_save(data['title'], 'title_tfidf_12')
tfidf_fit_save(data['title'], 'title_tfidf_1', (1,1))
tfidf_fit_save(data['title'], 'title_tfidf_2', (2,2))
tfidf_fit_save(data['full_text'], 'full_tfidf_12')
tfidf_fit_save(data['full_text'], 'full_tfidf_1', (1,1))
tfidf_fit_save(data['full_text'], 'full_tfidf_2', (2,2))

In [ ]:
tfidf_fit_save(data_preproc['title'], 'title_tfidf_12_preproc')
tfidf_fit_save(data_preproc['title'], 'title_tfidf_1_preproc', (1,1))
tfidf_fit_save(data_preproc['title'], 'title_tfidf_2_preproc', (2,2))
tfidf_fit_save(data_preproc['full_text'], 'full_tfidf_12_preproc')
tfidf_fit_save(data_preproc['full_text'], 'full_tfidf_1_preproc', (1,1))
tfidf_fit_save(data_preproc['full_text'], 'full_tfidf_2_preproc', (2,2))

### 2. Pretrained models

Download and save different types of models and tokenizers

In [ ]:
ruBERT_download_save()

#### 2.1 ruBERT

Make embeddings for all types of models

In [ ]:
model, tokenizer = ruBERT_load(name='ruBERT_tiny')
model.to('cuda')
all_embeddings = []
for i in tqdm(range(0,len(data))):
    all_embeddings.append(make_embeddings(data.iloc[i,:]['full_text'], model, tokenizer))

np.save('../embeddings/embeddings_ruBERT_tiny', np.array(all_embeddings))

all_embeddings = []
for i in tqdm(range(0,len(data_preproc))):
    all_embeddings.append(make_embeddings(data_preproc.iloc[i,:]['full_text'], model, tokenizer))

np.save('../embeddings/embeddings_ruBERT_tiny_preproc', np.array(all_embeddings))

In [ ]:
model, tokenizer = ruBERT_load(name='ruBERT_base')
model.to('cuda')
all_embeddings = []
for i in tqdm(range(0,len(data))):
    all_embeddings.append(make_embeddings(data.iloc[i,:]['full_text'], model, tokenizer))

np.save('../embeddings/embeddings_ruBERT_base', np.array(all_embeddings))

all_embeddings = []
for i in tqdm(range(0,len(data_preproc))):
    all_embeddings.append(make_embeddings(data_preproc.iloc[i,:]['full_text'], model, tokenizer))

np.save('../embeddings/embeddings_ruBERT_base_preproc', np.array(all_embeddings))

In [ ]:
model, tokenizer = ruBERT_load(name='ruBERT_threeway')
model.to('cuda')
all_embeddings = []
for i in tqdm(range(0,len(data))):
    all_embeddings.append(make_embeddings(data.iloc[i,:]['full_text'], model, tokenizer))

np.save('../embeddings/embeddings_ruBERT_threeway', np.array(all_embeddings))

all_embeddings = []
for i in tqdm(range(0,len(data_preproc))):
    all_embeddings.append(make_embeddings(data_preproc.iloc[i,:]['full_text'], model, tokenizer))

np.save('../embeddings/embeddings_ruBERT_threeway_preproc', np.array(all_embeddings))

Mounted at /content/drive


In [ ]:
model, tokenizer = ruBERT_load(name='ruBERT_tiny_bilingual')
model.to('cuda')
all_embeddings = []
for i in tqdm(range(0,len(data))):
    all_embeddings.append(make_embeddings(data.iloc[i,:]['full_text'], model, tokenizer))

np.save('../embeddings/embeddings_ruBERT_tiny_bilingual', np.array(all_embeddings))

all_embeddings = []
for i in tqdm(range(0,len(data_preproc))):
    all_embeddings.append(make_embeddings(data_preproc.iloc[i,:]['full_text'], model, tokenizer))

np.save('../embeddings/embeddings_ruBERT_tiny_bilingual', np.array(all_embeddings))

#### 2.2 LaBSE (Multilingual Embedding Model)

In [ ]:
model = SentenceTransformer('sentence-transformers/LaBSE')
all_embeddings = model.encode(data['full_text'], show_progress_bar=True)

np.save('../embeddings/LaBSE', np.array(all_embeddings))

all_embeddings = model.encode(data['full_text'], show_progress_bar=True)

np.save('../embeddings/LaBSE_preproc', np.array(all_embeddings))

### 3. Experiment

#### 3.1 TF_IDF recomendations

##### 3.1.1 Text without preprocessing

In [31]:
model_title_tfidf_12, matrix_title_tfidf_12 = tfidf_load_model('title_tfidf_12')
model_title_tfidf_1, matrix_title_tfidf_1 = tfidf_load_model('title_tfidf_1')
model_title_tfidf_2, matrix_title_tfidf_2 = tfidf_load_model('title_tfidf_2')
model_full_tfidf_12, matrix_full_tfidf_12 = tfidf_load_model('full_tfidf_12')
model_full_tfidf_1, matrix_full_tfidf_1 = tfidf_load_model('full_tfidf_1')
model_full_tfidf_2, matrix_full_tfidf_2 = tfidf_load_model('full_tfidf_2')

model_title_tfidf_12_preproc, matrix_title_tfidf_12_preproc = tfidf_load_model('title_tfidf_12_preproc')
model_title_tfidf_1_preproc, matrix_title_tfidf_1_preproc = tfidf_load_model('title_tfidf_1_preproc')
model_title_tfidf_2_preproc, matrix_title_tfidf_2_preproc = tfidf_load_model('title_tfidf_2_preproc')
model_full_tfidf_12_preproc, matrix_full_tfidf_12_preproc = tfidf_load_model('full_tfidf_12_preproc')
model_full_tfidf_1_preproc, matrix_full_tfidf_1_preproc = tfidf_load_model('full_tfidf_1_preproc')
model_full_tfidf_2_preproc, matrix_full_tfidf_2_preproc = tfidf_load_model('full_tfidf_2_preproc')

In [48]:
make_tfidf_recomendation(matrix_title_tfidf_12, 'инженерное налогообложение', model_title_tfidf_12, data, data_orig)

,title,description
0,Двойное налогообложение в России,Российские компании активно работают на зарубе...
1,инженерное будущее: создание удивительного,"узнайте, какие ключевые навыки и знания, котор..."
2,добыча в целях развития: налогообложение рычаж...,"налоговые доходы, скорее всего, будет основным..."
3,Налоги и налогообложение: специальные налоговы...,NaN
4,"Промышленность, инженерное образование и наука...",<p><br></p>\r\n
5,"Промышленность, инженерное образование и наука...",NaN
6,"Что такое ETF: инвестиции, особенности, разнов...",Разумная диверсификация активов помогает распр...
7,Защита мира: Вводя коррозии и инженерное дело,NaN
8,Открытие ВКС-конференции &quot;Современное инж...,
9,Логистический оператор Alibaba запустил в Кита...,"Беспилотники сами прокладывают маршрут, но пок..."


In [49]:
make_tfidf_recomendation(matrix_full_tfidf_12, 'инженерное налогообложение', model_full_tfidf_12, data, data_orig)

False

In [50]:
make_tfidf_recomendation(matrix_title_tfidf_1, 'инженерное налогообложение', model_title_tfidf_1, data, data_orig)

,title,description
0,Двойное налогообложение в России,Российские компании активно работают на зарубе...
1,инженерное будущее: создание удивительного,"узнайте, какие ключевые навыки и знания, котор..."
2,"Промышленность, инженерное образование и наука...",<p><br></p>\r\n
3,"Промышленность, инженерное образование и наука...",NaN
4,добыча в целях развития: налогообложение рычаж...,"налоговые доходы, скорее всего, будет основным..."
5,Налоги и налогообложение: специальные налоговы...,NaN
6,"Что такое ETF: инвестиции, особенности, разнов...",Разумная диверсификация активов помогает распр...
7,Защита мира: Вводя коррозии и инженерное дело,NaN
8,Открытие ВКС-конференции &quot;Современное инж...,
9,Логистический оператор Alibaba запустил в Кита...,"Беспилотники сами прокладывают маршрут, но пок..."


In [51]:
make_tfidf_recomendation(matrix_full_tfidf_1, 'инженерное налогообложение', model_full_tfidf_1, data, data_orig)

,title,description
0,"Промышленность, инженерное образование и наука...",NaN
1,"Промышленность, инженерное образование и наука...",<p><br></p>\r\n
2,Налоги и налогообложение: специальные налоговы...,NaN
3,Защита мира: Вводя коррозии и инженерное дело,NaN
4,Открытие ВКС-конференции &quot;Современное инж...,
5,Инженерное образование школьников: когда начин...,Когда нужно начинать инженерное образование де...
6,"Что такое ETF: инвестиции, особенности, разнов...",Разумная диверсификация активов помогает распр...
7,Отсрочка маркировки обуви: как работать до 1 и...,"Правительство дало обувникам квартал на то, чт..."
8,Двойное налогообложение в России,Российские компании активно работают на зарубе...
9,Мехатроника и робототехника,<p>Современное инженерное образование расширяе...


In [52]:
make_tfidf_recomendation(matrix_title_tfidf_2, 'инженерное налогообложение', model_title_tfidf_2, data, data_orig)

False

In [53]:
make_tfidf_recomendation(matrix_full_tfidf_2, 'инженерное налогообложение', model_full_tfidf_2, data, data_orig)

False

##### 3.1.2 Text with preprocessing

In [54]:
make_tfidf_recomendation(matrix_title_tfidf_12_preproc, preprocessing('инженерное налогообложение'), model_title_tfidf_12_preproc, data_preproc, data_orig)

,title,description
0,Двойное налогообложение в России,Российские компании активно работают на зарубе...
1,Основы налогообложения бизнеса в России,NaN
2,инженерная механика,вы заинтересованы в улучшении ваших механиков ...
3,Инженерная механика,"В курсе наглядно, в сочетании с математической..."
4,Инженерная механика,"В курсе наглядно, в сочетании с математической..."
5,инженерная механика,"это базовый курс первого уровня, чтобы узнать ..."
6,инженерная математика ii,"этот курс об основной математике, является фун..."
7,инженерная математика - i,"этот курс об основной математике, является фун..."
8,инженерная графика,основной целью данного курса является: студент...
9,Налогообложение I хозяйствующих субъектов: Кор...,NaN


In [55]:
make_tfidf_recomendation(matrix_full_tfidf_12_preproc, preprocessing('инженерное налогообложение'), model_full_tfidf_12_preproc, data_preproc, data_orig)

,title,description
0,Основы налогообложения бизнеса в России,NaN
1,Налогообложение I хозяйствующих субъектов: Кор...,NaN
2,Налоги и налогообложение: специальные налоговы...,NaN
3,Налогообложение хозяйствующих субъектов II: Pa...,NaN
4,Упрощенная система налогообложения станет еще ...,Госдумой в первом чтении приняты проекты закон...
5,Налогообложение в Российской Федерации: основа...,NaN
6,Введение в инженерные механик,NaN
7,Применение в инженерной механики,NaN
8,Сэкономить на налогах — советы главного бухгал...,Пригодятся всем компаниям на основной системе ...
9,Двойное налогообложение в России,Российские компании активно работают на зарубе...


In [ ]:
make_tfidf_recomendation(matrix_title_tfidf_2_preproc, preprocessing('инженерное налогообложение'), model_title_tfidf_2_preproc, data_preproc, data_orig)

False

In [ ]:
make_tfidf_recomendation(matrix_full_tfidf_2_preproc, preprocessing('инженерное налогообложение'), model_full_tfidf_2_preproc, data_preproc, data_orig)

False

In [56]:
make_tfidf_recomendation(matrix_title_tfidf_1_preproc, preprocessing('инженерное налогообложение'), model_title_tfidf_1_preproc, data_preproc, data_orig)

,title,description
0,Основы налогообложения бизнеса в России,NaN
1,Двойное налогообложение в России,Российские компании активно работают на зарубе...
2,инженерная математика ii,"этот курс об основной математике, является фун..."
3,инженерная математика - i,"этот курс об основной математике, является фун..."
4,инженерная механика,"это базовый курс первого уровня, чтобы узнать ..."
5,инженерная механика,вы заинтересованы в улучшении ваших механиков ...
6,Инженерная механика,"В курсе наглядно, в сочетании с математической..."
7,Инженерная механика,"В курсе наглядно, в сочетании с математической..."
8,Упрощенная система налогообложения станет еще ...,Госдумой в первом чтении приняты проекты закон...
9,инженерная графика,основной целью данного курса является: студент...


In [57]:
make_tfidf_recomendation(matrix_full_tfidf_1_preproc, preprocessing('инженерное налогообложение'), model_full_tfidf_1_preproc, data_preproc, data_orig)

,title,description
0,Основы налогообложения бизнеса в России,NaN
1,Налоги и налогообложение: специальные налоговы...,NaN
2,Упрощенная система налогообложения станет еще ...,Госдумой в первом чтении приняты проекты закон...
3,Налогообложение I хозяйствующих субъектов: Кор...,NaN
4,Управление Основные Инженерные проекты,NaN
5,Налогообложение в Российской Федерации: основа...,NaN
6,Налогообложение хозяйствующих субъектов II: Pa...,NaN
7,Введение в инженерные механик,NaN
8,Сэкономить на налогах — советы главного бухгал...,Пригодятся всем компаниям на основной системе ...
9,Применение в инженерной механики,NaN


In [58]:
model, tokenizer = ruBERT_load('ruBERT_tiny')
all_embeddings = np.load('../embeddings/embeddings_ruBERT_tiny.npy')
make_recomendation(all_embeddings, 'инженерное налогообложение', model, tokenizer, data, data_orig)

,title,description
0,Образовательный data engineering,NaN
1,Инновационный менеджмент,NaN
2,Поведенческие Инвестирование,NaN
3,Статистические выводы,NaN
4,Наука Осуществления,NaN
5,Проектное финансирование,NaN
6,Финансовые технологии,NaN
7,Финансовые технологии,<p><br></p>\r\n
8,Развитие Drug,NaN
9,Интеллектуальная обработка,NaN


In [ ]:
model, tokenizer = ruBERT_load('ruBERT_tiny')
all_embeddings = np.load('../embeddings/embeddings_ruBERT_tiny_preproc.npy')
make_recomendation(all_embeddings, 'инженерное налогообложение', model, tokenizer, data, data_orig)

In [75]:
model, tokenizer = ruBERT_load('ruBERT_threeway')
all_embeddings = np.load('../embeddings/embeddings_ruBERT_threeway.npy')
make_recomendation(all_embeddings, 'инженерное налогообложение', model, tokenizer, data, data_orig)

,title,description
0,Разбор стратегии автомарафона на 1 млн с запу...,NaN
1,Налоговое право (Tax law),NaN
2,"Кейс продвижения горчичного масла ""Горлинка""",NaN
3,Современная комбинаторика (Modern combinatorics),NaN
4,Эконометрика (Econometrics),NaN
5,"Как растить маркетинг, не увеличивая стоимость...",NaN
6,Введение ускорителей частиц (НПББ Mooc),NaN
7,Макроэкономическое планирование и прогнозирова...,NaN
8,Как развиваться в области ux/ui ?,Какие ресурсы есть для развития
9,Строительство R пакеты,NaN


In [83]:
model, tokenizer = ruBERT_load('ruBERT_threeway')
all_embeddings = np.load('../embeddings/embeddings_ruBERT_threeway_preproc.npy')
make_recomendation(all_embeddings, 'инженерное налогообложение', model, tokenizer, data, data_orig)

,title,description
0,Финансовый анализ,NaN
1,Финансовый менеджмент,NaN
2,Финансовые технологии,<p><br></p>\r\n
3,Финансовые технологии,NaN
4,Финансовые рынки,NaN
5,Дизайн-мышление в инновационной деятельности,NaN
6,Электронная музыка Производство,NaN
7,Финансовый учет Основа,NaN
8,Финансовый учет: Основы,NaN
9,Тонкости верстки,NaN


In [77]:
model, tokenizer = ruBERT_load('ruBERT_tiny_bilingual')
all_embeddings = np.load('../embeddings/embeddings_ruBERT_tiny_biling.npy')
make_recomendation(all_embeddings, 'инженерное налогообложение', model, tokenizer, data, data_orig)

,title,description
0,Электролиз медного купороса,NaN
1,Финансовый анализ,NaN
2,Динамическое Манипулирование: Вводная лекция 2,
3,Слезивый котик и новый питомец)),Больше фотографий котика здесь: https://vk.com...
4,Машинное обучение Алгоритмы: Контролируемая об...,NaN
5,элон маск на страх,NaN
6,Формальный Финансовый учет,NaN
7,Финансовый отчет,NaN
8,байесовская статистика,NaN
9,Налоги и налогообложение: специальные налоговы...,NaN


In [85]:
model, tokenizer = ruBERT_load('ruBERT_tiny_bilingual')
all_embeddings = np.load('../embeddings/embeddings_ruBERT_tiny_biling_preproc.npy')
make_recomendation(all_embeddings, 'инженерное налогообложение', model, tokenizer, data, data_orig)

,title,description
0,Финансы для всех: Debt,NaN
1,Байесовская метода машинного обучения,NaN
2,"Карьера, команда, инвестиции. Во что инвестиро...",<p><strong>На митапе обсудим:</strong></p>\r\n...
3,Налоги и налогообложение: специальные налоговы...,NaN
4,fintech - финансовые инновации,"исследовать изменения, вызванные инновациями f..."
5,Почему бесплатный аудит сайта это плохо,NaN
6,Методологические проблемы психологии,NaN
7,Налогообложение в Российской Федерации: основа...,NaN
8,Язык и инструменты финансового анализа,NaN
9,Scholarly связи,NaN


In [87]:
model, tokenizer = ruBERT_load('ruBERT_base')
all_embeddings = np.load('../embeddings/embeddings_ruBERT_base.npy')
make_recomendation(all_embeddings, 'инженерное налогообложение', model, tokenizer, data, data_orig)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,title,description
0,Налоговое право (Tax law),NaN
1,Машинное обучение в финансах,NaN
2,Бюджетирование и ценообразование в ДПО,NaN
3,Налоговое право,<p><br></p>\r\n
4,Правовое обеспечение бизнеса в России,NaN
5,Финансовое планирование для молодых взрослых,NaN
6,Создание и управление активами,NaN
7,Международное управление портфелем,NaN
8,Юридическое оформление инвестиционных идей,NaN
9,Машинное обучение в бизнесе,NaN


In [89]:
all_embeddings = np.load('../embeddings/embeddings_ruBERT_base_preproc.npy')
make_recomendation(all_embeddings, 'инженерное налогообложение', model, tokenizer, data, data_orig)

,title,description
0,Финансовый менеджмент,NaN
1,Основы менеджмента,NaN
2,Основы менеджмента,NaN
3,Управление маркетингом я,NaN
4,Язык и инструменты финансового анализа,NaN
5,Территориальные механизмы поддержки для технол...,NaN
6,Финансовый анализ,NaN
7,Маркетинг в аналоговом мире,NaN
8,Контент Менеджмент,NaN
9,Анализ для бизнес-систем,NaN


In [91]:
model = LaBSE_load()
model.to('cpu')
all_embeddings = np.load('../embeddings/LaBSE.npy')
make_labse_recomendation(all_embeddings, 'инженерное налогообложение', model, data, data_orig, number=10)

,title,description
0,Искусство налогового планирования,NaN
1,Формальный Финансовый учет,NaN
2,Институциональная экономика (Institutional eco...,NaN
3,Математическое моделирование в инженерных науках,NaN
4,Машинное обучение в финансах,NaN
5,Финансовый отчет,NaN
6,Введение в инженерные механик,NaN
7,Программная инженерия,NaN
8,Финансовый анализ,NaN
9,Применение в инженерной механики,NaN


In [93]:
all_embeddings = np.load('../embeddings/LaBSE_preproc.npy')
make_labse_recomendation(all_embeddings, 'инженерное налогообложение', model, data, data_orig, number=10)

,title,description
0,Основы бухгалтерского учета,NaN
1,Формальный Финансовый учет,NaN
2,Теория и практика эффективной коммуникации,NaN
3,Финансовые технологии,<p><br></p>\r\n
4,Финансовые технологии,NaN
5,Учет и финансы для ИТ-специалистов,NaN
6,Анализ финансовой отчетности I: Роль бухгалтер...,NaN
7,Финансовый учет: Основы,NaN
8,Финансовый учет Основа,NaN
9,Управление проектами Инжиниринг,NaN


In [94]:
data_save =make_labse_recomendation(all_embeddings, 'инженерное налогообложение', model, data, data_orig, number=10)
data_save.to_csv('LaBSE_preproc.csv')

### Другие примеры запросов

In [64]:
model, tokenizer = ruBERT_load('ruBERT_base')
all_embeddings = np.load('../embeddings/embeddings_ruBERT_base.npy')
make_recomendation(all_embeddings, 'Как заработать денег не напрягаясь', model, tokenizer, data, data_orig)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,title,description
0,Как конфликт сделать прибыльным?,NaN
1,Как запустить зарубежные продажи не выходя из ...,NaN
2,Как выделиться на переполненном рынке,NaN
3,Как навести порядок в большом бизнесе с помощь...,NaN
4,Куда идти за инвестициями обычному предпринима...,NaN
5,Как начать свой бизнес,NaN
6,Как выбрать ментора?,NaN
7,Как оборудовать офис и не переплатить за железо,NaN
8,"Как сдать PMP, не выходя из дома",NaN
9,Как сделать сделку с корпорацией и не прогореть,NaN


In [65]:
all_embeddings = np.load('../embeddings/embeddings_ruBERT_base_preproc.npy')
make_recomendation(all_embeddings, 'Как заработать денег не напрягаясь', model, tokenizer, data, data_orig)

,title,description
0,Как начать свой бизнес,NaN
1,Как я выбираю акции в свой пенсионный портфель,NaN
2,Как построить бизнес в фармацевтике,NaN
3,Участие и Воспитывает стратегия маркетинга,NaN
4,ГеоДизайн: Изменить Ваш мир,NaN
5,Как вы привлекли своего первого клиента?,NaN
6,Как финансировать ваше предприятие?,NaN
7,Как меняется бизнес в условиях пандемии?,NaN
8,Как потребители воспринимают бренд?,NaN
9,"Как проверить идею до того, как вложиться в ра...",NaN


In [66]:
model, tokenizer = ruBERT_load('ruBERT_threeway')
all_embeddings = np.load('../embeddings/embeddings_ruBERT_threeway.npy')
make_recomendation(all_embeddings, 'Как заработать денег не напрягаясь', model, tokenizer, data, data_orig)

,title,description
0,Как организовать работу дома с 2 детьми и не с...,NaN
1,Какой бизнес нужен Иннополису?,Кто может заработать на развитии будущей ИТ-ст...
2,Как запустить зарубежные продажи не выходя из ...,NaN
3,Разбор стратегии автомарафона на 1 млн с запу...,NaN
4,"Как стать участником проекта ""Сколково""",NaN
5,Как повысить удержание на вебинаре до 85-90%,Личный опыт.
6,Как с помощью онлайн-консультанта увеличить за...,NaN
7,"Кейс продвижения горчичного масла ""Горлинка""",NaN
8,"Бизнес-секреты ""Лед тронулся""",NaN
9,«Аренда лофтов в Твери»: можно ли заработать н...,NaN


In [67]:
model, tokenizer = ruBERT_load('ruBERT_base')
all_embeddings = np.load('../embeddings/embeddings_ruBERT_base.npy')
make_recomendation(all_embeddings, 'изучить линейную алгебру', model, tokenizer, data, data_orig)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,title,description
0,Линейная алгебра: от идеи к формуле,NaN
1,Последовательность моделей для временных рядов...,NaN
2,Применение производной и интеграла в курсе общ...,NaN
3,введение в дискретной математике,"в этом курсе, посвященном основам математики, ..."
4,Смешанные линейные модели,NaN
5,Как сделать гипотезы связующим звеном для всех...,NaN
6,Практический анализ временных рядов,NaN
7,SQL для науки данных,NaN
8,Обучение на размеченных данных,NaN
9,"Абстракция, проблема декомпозиции, и функции",NaN


In [68]:
all_embeddings = np.load('../embeddings/embeddings_ruBERT_base_preproc.npy')
make_recomendation(all_embeddings, 'изучить линейную алгебру', model, tokenizer, data, data_orig)

,title,description
0,Матрица Алгебра для инженеров,NaN
1,Алгоритмы программирования и структуры данных,<p><br></p>\r\n
2,Вектор Исчисление для инженеров,NaN
3,Усовершенствованные алгоритмы и сложность,NaN
4,Сети Иллюстрированные: Принципы без Исчисление,NaN
5,Основы программирования на R,<p><br></p>\r\n
6,Введение в перечислительной комбинаторики,NaN
7,Основы программирования на C. Задачи.,NaN
8,"Моделирование, анализ алгоритма и указатели",NaN
9,Шесть Сигма Расширенный анализ Фазу,NaN


In [69]:
model, tokenizer = ruBERT_load('ruBERT_threeway')
all_embeddings = np.load('../embeddings/embeddings_ruBERT_threeway.npy')
make_recomendation(all_embeddings, 'изучить линейную алгебру', model, tokenizer, data, data_orig)

,title,description
0,Оценить дизайн,Уважаемые коллеги!
1,Просто о статистике (с использованием R),NaN
2,Знакомимся с основами HubSpot,NaN
3,Более общее преобразование Лоренца,NaN
4,Подумайте снова я: Как понять аргументы,NaN
5,Добро пожаловать в теории игр,NaN
6,Секреты принятия правильных решений в условиях...,NaN
7,Подумайте еще раз II: Как Причина дедуктивным,NaN
8,Аппроксимация алгоритмы Часть I,NaN
9,Шесть Сигма Расширенный определения и измерени...,NaN


In [70]:
model, tokenizer = ruBERT_load('ruBERT_base')
all_embeddings = np.load('../embeddings/embeddings_ruBERT_base.npy')
make_recomendation(all_embeddings, 'что бы почитать', model, tokenizer, data, data_orig)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,title,description
0,"Что посмотреть, почитать и послушать для опера...",NaN
1,Как относиться к жизни,NaN
2,"Что нужно делать, чтобы не выгореть",NaN
3,Как оборудовать офис и не переплатить за железо,NaN
4,Как начать свой бизнес,NaN
5,"Что почитать, когда только собираешься по пути...","Здравствуйте, человеки."
6,Как читать фотографию,NaN
7,Как проверить вашу идею запуска,NaN
8,Просто про страхование,NaN
9,"Как сдать PMP, не выходя из дома",NaN


In [71]:
all_embeddings = np.load('../embeddings/embeddings_ruBERT_base_preproc.npy')
make_recomendation(all_embeddings, 'что бы почитать', model, tokenizer, data, data_orig)

,title,description
0,Читали? Что полезного вынесли?,NaN
1,Как читать фотографию,NaN
2,"Хорошо, отдыхай",NaN
3,Подумайте снова я: Как понять аргументы,NaN
4,Где живет интернет? (Фото),NaN
5,Что такое современное искусство?,NaN
6,"Как проверить идею до того, как вложиться в ра...",NaN
7,Как меняется бизнес в условиях пандемии?,NaN
8,Сколько стоит подождать,NaN
9,Серия 3. «Чем делать?»,NaN


In [72]:
model, tokenizer = ruBERT_load('ruBERT_threeway')
all_embeddings = np.load('../embeddings/embeddings_ruBERT_threeway.npy')
make_recomendation(all_embeddings, 'что бы почитать', model, tokenizer, data, data_orig)

,title,description
0,Что за подсайт?,NaN
1,Баг с галочкой,Здравствуйте.
2,Введение в С++,NaN
3,Что за зверь?,NaN
4,"Кайботт, ""Паркетчики""","Caillebotte, The Floor Scrapers"
5,"Бизнес-секреты ""Лед тронулся""",NaN
6,Ключи от Доброго Углича,NaN
7,Что нового?,NaN
8,Какой бизнес нужен Иннополису?,Кто может заработать на развитии будущей ИТ-ст...
9,"Кайботт, ""Мужчина в ванной""","Caillebotte, Man at his Bath"
